In [1]:
import numpy as np
import numpy.testing as npt
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.stats
import logging

In [2]:
import gym
env = gym.make('FrozenLake-v0' )

[2017-12-12 16:25:49,598] Making new env: FrozenLake-v0


In [3]:
state  = env.reset()

In [4]:
def act(action):
    state, reward, done, info = env.step(action)
    print (state)
    env.render()  

In [5]:
act(3)

0
  (Up)
SFFF
FHFH
FFFH
HFFG


In [6]:
pi = {0:1, 1:2, 2:1, 3:0, 4:1, 6:1, 8:2, 9:0, 10:1, 13:2, 14:2}

In [7]:
state  = env.reset()
print (state)
done = False
while not done:
    env.render()
    action = pi[state]
    print ("random action:", action)
    state, reward, done, info = env.step(action)
    print (state, reward, done, info)
    if done:
        print ("return:", reward) # return for all visited states is here last reward

env.render(close=True)

0

SFFF
FHFH
FFFH
HFFG
random action: 1
4 0.0 False {'prob': 0.3333333333333333}
  (Down)
SFFF
FHFH
FFFH
HFFG
random action: 1
5 0.0 True {'prob': 0.3333333333333333}
return: 0.0


In [8]:
policy = lambda a: pi[a]